In [51]:
%load_ext autoreload
%autoreload 2

# Standard libraries
import logging
from os.path import join

# Libraries for graphs
import community
import networkx as nx

# Libraries for matrix computations
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import numpy.matlib as matlib
import pandas as pd

# Libraries for sparse matrices and eigenvectors
from scipy.sparse import diags
from scipy.sparse.linalg import eigs
from scipy.stats import hypergeom, pearsonr

# Import functions from nibna module
from nibna import build_graph_from_edge_list, build_node_importance_dataframe, compute_importance_score_threshold, \
compute_eigenvectors, compute_node_importance, compute_pearson_correlation, configure_logging, NUM_miR, \
perform_community_detection, validate_top_k_coding_genes

# configure_logging()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
ROOT_DIR = "/Users/mandar.chaudhary/Research Wednesday/CancerDriver/"
DATA_DIR = "/Users/mandar.chaudhary/Research Wednesday/NIBNA/Data/"
SUBTYPE_DATA_DIR = "/Users/mandar.chaudhary/Research Wednesday/CancerDriver/Data/Output/CancerSubtype"

## Initialize cancer subtypes

In [28]:
subtypes = ["Basal", "Her2", "LumA", "LumB", "Normal"]

## Perform NIBNA on each cancer subtype

In [30]:
cancer_network_filename = 'pVal_cancer_network.csv'
for subtype in subtypes:
    logger.info('Subtype: {0}'.format(subtype))
    # Read Mes data and network
    filename = subtype.lower() + '_cancer_data.csv'
    subtype_df = pd.read_csv(join(SUBTYPE_DATA_DIR, subtype, filename))
    subtype_network = pd.read_csv(join(SUBTYPE_DATA_DIR, subtype, cancer_network_filename))
    logger.info('Dataframe size: {0}'.format(subtype_df.shape))
    logger.info('Computing pearson correlation')
    subtype_network = compute_pearson_correlation(subtype_network, subtype_df)
    
    logger.info('Building graph from edge list')
    G = build_graph_from_edge_list(subtype_network, is_weighted=True)
    
    np.random.seed(10)
    n_communities = perform_community_detection(G)
    adj_eigen_vectors = compute_eigenvectors(G, n_communities)
    
    logger.info('Computing node importance')
    I = compute_node_importance(G.number_of_nodes(), n_communities, np.real(adj_eigen_vectors))
    I_df = build_node_importance_dataframe(G, I)
    
    cutoff_threshold = compute_importance_score_threshold(I_df.importance.values)
    logger.info('Cutoff threshold: {0}'.format(cutoff_threshold))
    
    critical_nodes = pd.DataFrame(I_df.node.iloc[:cutoff_threshold])
    
    critical_nodes["Type"] = "coding"
    critical_nodes.loc[critical_nodes.node.isin(subtype_df.columns[:NUM_miR]), "Type"] = "non-coding"
    
    critical_nodes.to_csv(join(DATA_DIR, subtype, 'critical_nodes.csv'))

INFO:root:Subtype: Basal
INFO:root:Dataframe size: (158, 7726)
INFO:root:Computing pearson correlation
INFO:root:Building graph from edge list
INFO:nibna:Number of nodes: 5927, and number of edges: 28654
INFO:nibna:Graph edge weights: 9521.478279272036
INFO:root:Computing node importance
INFO:nibna:Number of communities in G: 44
INFO:root:Cutoff threshold: 271
INFO:root:Subtype: Her2
INFO:root:Dataframe size: (108, 7726)
INFO:root:Computing pearson correlation
INFO:root:Building graph from edge list
INFO:nibna:Number of nodes: 4720, and number of edges: 11972
INFO:nibna:Graph edge weights: 4828.463222880856
INFO:root:Computing node importance
INFO:nibna:Number of communities in G: 113
INFO:root:Cutoff threshold: 532
INFO:root:Subtype: LumA
INFO:root:Dataframe size: (221, 7726)
INFO:root:Computing pearson correlation
INFO:root:Building graph from edge list
INFO:nibna:Number of nodes: 6341, and number of edges: 43998
INFO:nibna:Graph edge weights: 13274.756588130405
INFO:root:Computing n

## Load gold standard data for validation

In [48]:
mRNAs_df = pd.read_csv(join(DATA_DIR, "mRNAs_data.csv"))

In [49]:
gold_standard_cgc_df = pd.read_csv(join(DATA_DIR, "Census_allFri Sep 28 07_39_37 2018.tsv"), sep="\t")
gold_standard_cgc = gold_standard_cgc_df["Gene Symbol"]

## Validate coding genes estimated by NIBNA

In [81]:
for subtype in subtypes:
    subtype_critical_nodes = pd.read_csv(join(SUBTYPE_DATA_DIR, subtype, "critical_nodes.csv"))
    subtype_critical_coding_nodes = subtype_critical_nodes.loc[subtype_critical_nodes.node.isin(mRNAs_df.columns), ].copy()
    subtype_critical_coding_nodes['In CGC'] = 'No'
    subtype_critical_coding_nodes.loc[subtype_critical_coding_nodes.node.isin(gold_standard_cgc), 'In CGC'] = 'Yes'
    logger.info('Cancer subtype: {0}'.format(subtype))
    logger.info('# estimated drivers: {0}, # validated coding genes: {1}'.format(len(subtype_critical_nodes), (subtype_critical_coding_nodes['In CGC']=='Yes').sum()))
    n_A = len(subtype_critical_nodes) # number of estimated drivers
    n_B = len(gold_standard_cgc) # number of signatures
    n_C = 839+5168 # number of genes
    n_A_B = (subtype_critical_coding_nodes['In CGC']=='Yes').sum() # number of drivers validated
    p_value = 1 - hypergeom.cdf(n_A_B, n_C, n_B, n_A)
    logger.info("p_value for coding drivers: {0}".format(p_value))

INFO:root:Cancer subtype: Basal
INFO:root:# estimated drivers: 271, # validated coding genes: 68
INFO:root:p_value for coding drivers: 2.967787127161614e-10
INFO:root:Cancer subtype: Her2
INFO:root:# estimated drivers: 532, # validated coding genes: 120
INFO:root:p_value for coding drivers: 9.249045973547254e-12
INFO:root:Cancer subtype: LumA
INFO:root:# estimated drivers: 236, # validated coding genes: 56
INFO:root:p_value for coding drivers: 8.294983333545503e-08
INFO:root:Cancer subtype: LumB
INFO:root:# estimated drivers: 379, # validated coding genes: 86
INFO:root:p_value for coding drivers: 4.113109852710295e-10
INFO:root:Cancer subtype: Normal
INFO:root:# estimated drivers: 471, # validated coding genes: 101
INFO:root:p_value for coding drivers: 3.731110975735419e-10


In [73]:
len(subtype_critical_nodes), len(subtype_critical_coding_nodes)

(471, 438)